# Empire PowerShell Launch Parameters
Detects suspicious powershell command line parameters used in Empire

## Rule Content
```
- title: Empire PowerShell Launch Parameters
  id: 79f4ede3-402e-41c8-bc3e-ebbf5f162581
  description: Detects suspicious powershell command line parameters used in Empire
  status: experimental
  references:
  - https://github.com/EmpireProject/Empire/blob/c2ba61ca8d2031dad0cfc1d5770ba723e8b710db/lib/common/helpers.py#L165
  - https://github.com/EmpireProject/Empire/blob/e37fb2eef8ff8f5a0a689f1589f424906fe13055/lib/modules/powershell/persistence/powerbreach/deaduser.py#L191
  - https://github.com/EmpireProject/Empire/blob/e37fb2eef8ff8f5a0a689f1589f424906fe13055/lib/modules/powershell/persistence/powerbreach/resolver.py#L178
  - https://github.com/EmpireProject/Empire/blob/e37fb2eef8ff8f5a0a689f1589f424906fe13055/data/module_source/privesc/Invoke-EventVwrBypass.ps1#L64
  author: Florian Roth
  date: 2019/04/20
  tags:
  - attack.execution
  - attack.t1086
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - '* -NoP -sta -NonI -W Hidden -Enc *'
      - '* -noP -sta -w 1 -enc *'
      - '* -NoP -NonI -W Hidden -enc *'
    condition: selection
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*\ \-NoP\ \-sta\ \-NonI\ \-W\ Hidden\ \-Enc\ * OR *\ \-noP\ \-sta\ \-w\ 1\ \-enc\ * OR *\ \-NoP\ \-NonI\ \-W\ Hidden\ \-enc\ *)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()